Today, Chancellor of the Exchequer Jeremy Hunt announced the Government's Spring Budget 2024. This budget includes a variety of reforms, delineated by the Treasury in LINK.

In this post, PolicyEngine will model the budgetary, societal, and distributional impacts of the budget's reforms to National Insurance, the High Income Child Benefit Charge, and the fuel duty.

[LINK TO MODEL FOR HOUSEHOLD]

## Spring Budget 2024 reforms
The Spring 2024 Budget includes reforms to various different segments of the tax code. Of these, PolicyEngine will be modelling the following in this article:

#### National Insurance Contributions (NICs)
* Lowering the main rate of Class 1 NICs from 10p to 8p
* Cutting the main rate of Class 4 self-employed NICs from 8p to 6p

#### High Income Child Benefit Charge
* Increasing the income threshold for the charge from £60,000 to £80,000
* Increasing the charge's taper range from £60,000 to £80,000

#### Fuel Duty
* Extending the fuel duty's 5p rate cut for 12 months
* Forgoing the fiscal year 2024-2025 RPI increase for the fuel duty

When modelling these reforms, PolicyEngine assumes they take effect on 1 January 2024 and evaluates their impacts over calendar year 2024, differing slightly from Treasury's estimates, which are aligned with the fiscal calendar. In future development iterations, PolicyEngine hopes to be able to model the mid-year effective date of reforms like these.

#### Other reforms
The Chancellor's Spring Budget 2024 also codifies a number of reforms not included in the PolicyEngine model, such as a freeze on the alcohol duty, changes to the non-domiciled taxation regime, and a duty on vaping products. These reforms will not be modelled in this article.

## Reform outcomes
Table 1 below presents PolicyEngine's estimate of each policy reform provision's key household, budgetary, and societal impact metrics. Cumulatively, the reforms are modelled to cost HM Treasury a total of £14.1 billion for calendar year 2024 and raise the net income of 78.9% of British households, particularly those with higher pre-tax income. The policies also present ambiguous impacts upon the poverty rate, while raising inequality by 0.4%.

*Table 1: Household and societal impacts of each Spring Budget 2023 reform provision*

In [1]:
%pip install -U policyengine_uk
%pip install -U pandas


  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from policyengine_uk import Microsimulation
from policyengine_uk.model_api import *

reform = {
    "gov.hmrc.fuel_duty.petrol_and_diesel": {
        "year:2024:5": 0.5295,
    },
    "gov.hmrc.income_tax.charges.CB_HITC.phase_out_start": {
        "year:2024:5": 60_000,
    },
    "gov.hmrc.income_tax.charges.CB_HITC.phase_out_rate": {
        "year:2024:5": 0.05,
    },
    "gov.hmrc.national_insurance.class_1.rates.employee.main": {
        "year:2024:5": 0.08,
    },
    "gov.hmrc.national_insurance.class_4.rates.main": {
        "year:2024:5": 0.06,
    },
}
import pandas as pd

# DataFrame with columns: [provision name, provision cost in 2024, percent better off, provision poverty impact in 2024, inequality impact in 2024]

reforms = []

fuel_duty_freeze = {
    "gov.hmrc.fuel_duty.petrol_and_diesel": {
        "year:2024:5": 0.5295,
    }
}

cb_hitc_cut = {
    "gov.hmrc.income_tax.charges.CB_HITC.phase_out_start": {
        "year:2024:5": 60_000,
    },
    "gov.hmrc.income_tax.charges.CB_HITC.phase_out_rate": {
        "year:2024:5": 0.05,
    },
}

ni_class_1_cut = {
    "gov.hmrc.national_insurance.class_1.rates.employee.main": {
        "year:2024:5": 0.08,
    },
}

ni_class_4_cut = {
    "gov.hmrc.national_insurance.class_4.rates.main": {
        "year:2024:5": 0.06,
    },
}

complete_reform = {
    **fuel_duty_freeze,
    **cb_hitc_cut,
    **ni_class_1_cut,
    **ni_class_4_cut
}

reforms = [fuel_duty_freeze, cb_hitc_cut, ni_class_1_cut, ni_class_4_cut, complete_reform]

provisions = ["Fuel duty freeze", "CB HITC cut", "NI class 1 cut", "NI class 4 cut", "Combined"]

baseline = Microsimulation()

def get_impacts(reform_dict: dict) -> dict:
    reform = Reform.from_dict(reform_dict, country_id="uk")
    reformed = Microsimulation(reform=reform)

    budgetary_impact = baseline.calculate("household_net_income", period=2024).sum() - reformed.calculate("household_net_income", 2024).sum()
    poverty_baseline = baseline.calculate("in_poverty", map_to="person", period=2024).mean()
    poverty_reform = reformed.calculate("in_poverty", map_to="person", period=2024).mean()
    poverty_rate_change = (poverty_reform - poverty_baseline) / poverty_baseline

    gini_baseline = baseline.calculate("equiv_household_net_income", map_to="person", period=2024).gini()
    gini_reform = reformed.calculate("equiv_household_net_income", map_to="person", period=2024).gini()
    gini_rate_change = (gini_reform - gini_baseline) / gini_baseline

    hnet_reformed = reformed.calculate("household_net_income", period=2024, map_to="person")
    hnet_baseline = baseline.calculate("household_net_income", period=2024, map_to="person")
    percent_better_off = (hnet_reformed > hnet_baseline).mean()
    # Poverty headcount relative change
    # Inequality gini index relative change
    # Percent better off

    return dict(
        budgetary_impact=budgetary_impact,
        reform_id=reform.api_id,
        poverty_rate_change=poverty_rate_change,
        gini_rate_change=gini_rate_change,
        percent_better_off=percent_better_off
    )

budgetary_impacts = []
reform_ids = []
poverty_rate_changes = []
gini_rate_changes = []
percentages_better_off = []

for i in range(len(provisions)):
    impacts = get_impacts(reforms[i])
    budgetary_impacts.append(impacts["budgetary_impact"])
    reform_ids.append(impacts["reform_id"])
    poverty_rate_changes.append(impacts["poverty_rate_change"])
    gini_rate_changes.append(impacts["gini_rate_change"])
    percentages_better_off.append(impacts["percent_better_off"])

df = pd.DataFrame({
    "Reform": provisions,
    "Budgetary impact": budgetary_impacts,
    "Percent gaining": percentages_better_off,
    "Poverty change": poverty_rate_changes,
    "Inequality change": gini_rate_changes,
    "PolicyEngine": reform_ids,
})

df["Budgetary impact"] = df["Budgetary impact"].apply(lambda x: f"{x/1e9:+,.1f}")
df["PolicyEngine"] = df["PolicyEngine"].apply(lambda x: f"[#{x}](https://policyengine.org/uk/policy?reform={x}&baseline=1&time_period=2024&region=uk)")
df["Percent gaining"] =  df["Percent gaining"].apply(lambda x: f"{x:+.1%}")
df["Poverty change"] =  df["Poverty change"].apply(lambda x: f"{x:+.1%}")
df["Inequality change"] =  df["Inequality change"].apply(lambda x: f"{x:+.1%}")

In [7]:
from IPython.display import Markdown

Markdown(df.to_markdown(index=False))

| Reform           |   Budgetary impact | Percent gaining   | Poverty change   | Inequality change   | PolicyEngine                                                                                    |
|:-----------------|-------------------:|:------------------|:-----------------|:--------------------|:------------------------------------------------------------------------------------------------|
| Fuel duty freeze |               -1   | +37.6%            | -0.6%            | -0.0%               | [#49760](https://policyengine.org/uk/policy?reform=49760&baseline=1&time_period=2024&region=uk) |
| CB HITC cut      |               -1.4 | +7.0%             | +0.0%            | +0.0%               | [#49761](https://policyengine.org/uk/policy?reform=49761&baseline=1&time_period=2024&region=uk) |
| NI class 1 cut   |              -10   | +63.8%            | -0.6%            | +0.3%               | [#49643](https://policyengine.org/uk/policy?reform=49643&baseline=1&time_period=2024&region=uk) |
| NI class 4 cut   |               -1   | +8.0%             | +0.0%            | +0.0%               | [#49762](https://policyengine.org/uk/policy?reform=49762&baseline=1&time_period=2024&region=uk) |
| Combined         |              -13.4 | +76.9%            | -1.0%            | +0.3%               | [#49773](https://policyengine.org/uk/policy?reform=49773&baseline=1&time_period=2024&region=uk) |

In [8]:
reformed = Microsimulation()

## Budgetary impacts

The PolicyEngine microsimulation model projects that the six reforms covered in this article will cost a combined £14.1 billion in 2024 [LINK]. This is only slightly higher than the Treasury's projected cost of [£13.7 billion](https://assets.publishing.service.gov.uk/media/65e7920c08eef600155a5617/Published_Costing_Document_Spring_Budget_2024_Final.pdf) for the same six reforms in fiscal year 2024-2025. Some of the variance in these projections may be attributable to HM Treasury's inclusion of behavioural responses to policy reforms, as well as Treasury's modelling over a the fiscal year, as opposed to the calendar.

Table 2 below presents PolicyEngine's broader estimates over 2024, 2025, and 2026, as compared to HM Treasury's [estimates](https://assets.publishing.service.gov.uk/media/65e8578eb559930011ade2cb/E03057752_HMT_Spring_Budget_Mar_24_Web_Accessible__2_.pdf) over the corresponding fiscal years (2024-2025, etc.) for the six policies investigated in this article.

In [9]:
from policyengine_uk import Microsimulation
from policyengine_uk.model_api import *

import pandas as pd

# DataFrame with columns: [provision name, provision cost in 2024, percent better off, provision poverty impact in 2024, inequality impact in 2024]

# reforms = []
reforms = {
    "2024": {
        "gov.hmrc.fuel_duty.petrol_and_diesel": {
            "year:2024:1": 0.5295,
        },
        "gov.hmrc.income_tax.charges.CB_HITC.phase_out_start": {
            "year:2024:1": 60_000,
        },
        "gov.hmrc.income_tax.charges.CB_HITC.phase_out_rate": {
            "year:2024:1": 0.05,
        },
        "gov.hmrc.national_insurance.class_1.rates.employee.main": {
            "year:2024:1": 0.08,
        },
        "gov.hmrc.national_insurance.class_4.rates.main": {
            "year:2024:1": 0.06,
        },
    },
    "2025": {
        "gov.hmrc.fuel_duty.petrol_and_diesel": {
            "year:2025:1": 0.5295,
        },
        "gov.hmrc.income_tax.charges.CB_HITC.phase_out_start": {
            "year:2025:1": 60_000,
        },
        "gov.hmrc.income_tax.charges.CB_HITC.phase_out_rate": {
            "year:2025:1": 0.05,
        },
        "gov.hmrc.national_insurance.class_1.rates.employee.main": {
            "year:2025:1": 0.08,
        },
        "gov.hmrc.national_insurance.class_4.rates.main": {
            "year:2025:1": 0.06,
        },
    },
    "2026": {
        "gov.hmrc.fuel_duty.petrol_and_diesel": {
            "year:2026:1": 0.5295,
        },
        "gov.hmrc.income_tax.charges.CB_HITC.phase_out_start": {
            "year:2026:1": 60_000,
        },
        "gov.hmrc.income_tax.charges.CB_HITC.phase_out_rate": {
            "year:2026:1": 0.05,
        },
        "gov.hmrc.national_insurance.class_1.rates.employee.main": {
            "year:2026:1": 0.08,
        },
        "gov.hmrc.national_insurance.class_4.rates.main": {
            "year:2026:1": 0.06,
        },
    }
}

rows = ["PolicyEngine", "HM Treasury", "Relative difference"]

baseline = Microsimulation()

# Assign values for HM Treasury from report
treasury_values_bn = {
    "2024": {
        "ni_class_1_cut": -9.360,
        "ni_class_4_cut": -0.710,
        "cb_hitc_cut": -0.540,
        "fuel_duty_freeze": -3.090
    },
    "2025": {
        "ni_class_1_cut": -9.295,
        "ni_class_4_cut": -0.850,
        "cb_hitc_cut": -0.635,
        "fuel_duty_freeze": -0.820
    },
    "2026": {
        "ni_class_1_cut": -9.490,
        "ni_class_4_cut": -0.735,
        "cb_hitc_cut": -0.640,
        "fuel_duty_freeze": -0.830
    },
}

treasury_totals_bn = {}
for year in treasury_values_bn:
    treasury_totals_bn[year] = sum(treasury_values_bn[year].values())

# Get impacts of entire policy reform for each year and place into dict
def get_budget_impact_year(reform_dict: dict, year: str) -> float:
    reform = Reform.from_dict(reform_dict, country_id="uk")
    reformed = Microsimulation(reform=reform)

    budgetary_impact = baseline.calculate("household_net_income", period=year).sum() - reformed.calculate("household_net_income", year).sum()
    return budgetary_impact

pe_totals_bn = {}
for year in ["2024", "2025", "2026"]:
    pe_totals_bn[year] = get_budget_impact_year(reforms[year], year)/1e9

# Calculate difference between values
diff_totals_bn = {}

for year in ["2024", "2025", "2026"]:
    diff_totals_bn[year] = treasury_totals_bn[year] - pe_totals_bn[year]

df = pd.DataFrame.from_dict(
    data={
      "": ["2024", "2025", "2026"],
      rows[0]: pe_totals_bn.values(),
      rows[1]: treasury_totals_bn.values(),
      rows[2]: diff_totals_bn.values()
    },
)

df = df.transpose()

In [31]:
from IPython.display import Markdown

Markdown(df.to_markdown())

|                     |           0 |          1 |          2 |
|:--------------------|------------:|-----------:|-----------:|
|                     | 2024        | 2025       | 2026       |
| PolicyEngine        |  -13.4082   |  -14.8754  |  -15.3443  |
| HM Treasury         |  -13.7      |  -11.6     |  -11.695   |
| Relative difference |   -0.291787 |    3.27537 |    3.64935 |